In [26]:
 #Import All Libraries Here
import pandas as pd
from sklearn.metrics import accuracy_score ,  roc_curve, auc , classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np

import gensim
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split , RandomizedSearchCV
import xgboost as xgb 
import statistics
from scipy.fftpack import fft, ifft


from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import RandomOverSampler

from igraph import *

from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import optuna

import sklearn.datasets

# PyTorch Import

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [27]:
# Record Constants
INPUT_TRAIN_IN = "../../data/train_in.csv"
INPUT_TRAIN_OUT = "../../data/train_out.csv"  
INPUT_TEST_IN = "../../data/test_in.csv"
INPUT_TEST_OUT = "../../data/test_out.csv"
INPUT_VALIDATION_IN = "../../data/valid_in_nucleo.csv"
INPUT_VALIDATION_OUT  = "../../data/valid_out.csv"

TARGET_MODEL_PATH = '../../webapp/model_files'


WINDOW_SIZE =  50 # Final RNA Sequence will be 101 Length 
MAX_LENGTH = (WINDOW_SIZE*2) +1

# 1 - Multi Dimensional K-mer with One Hot 
ENCODING_METHOD = 1

# 1 - LSTM with Cross Entropy 
MODEL = 1

FRAMEWORK = "PYTORCH"

INPUT_DIMENSION = 0

PERFORM_DATA_BALANCING = False
K_MERS_SIZE = 3


# Startegy to Crop Sequene
# MID - Modification is present at Mid of cropped Sequence 
# END - Modification is present at End of cropepd Sequence 
CROP_STRATEGY = 'END'

# Y Category Encoding Method
# LABEL or ONE_HOT 
TARGET_ENCODING = 'ONE_HOT'


In [28]:
#Read X Varaibles and Y Varaibles

x_train_raw =  pd.read_csv(INPUT_TRAIN_IN, header=None , skiprows=1 )
y_train_raw =  pd.read_csv(INPUT_TRAIN_OUT, header=None , skiprows=1)

x_test_raw =  pd.read_csv(INPUT_TEST_IN, header=None , skiprows=1 )
y_test_raw =  pd.read_csv(INPUT_TEST_OUT, header=None , skiprows=1)

x_valid_raw =  pd.read_csv(INPUT_VALIDATION_IN, header=None , skiprows=1 )
y_valid_raw =  pd.read_csv(INPUT_VALIDATION_OUT, header=None , skiprows=1 )

x_data = pd.concat([x_train_raw, x_test_raw, x_valid_raw], axis=0, ignore_index=True)
y_data = pd.concat([y_train_raw, y_test_raw, y_valid_raw], axis=0, ignore_index=True)


middle_index = (x_train_raw.shape[1] // 2) + 1

if CROP_STRATEGY == 'MID':
    STRAT_INEDX =middle_index - WINDOW_SIZE -1 
    END_INDEX =middle_index + WINDOW_SIZE 

if CROP_STRATEGY == 'END':
    STRAT_INEDX =middle_index - (WINDOW_SIZE*2) -1 
    END_INDEX =middle_index

# Testing data sample
# x_data =x_data.iloc[:10000]
# y_data =y_data.iloc[:10000]

print(f"Shape of X : {x_data.shape} and Shape of Y : {y_data.shape}")

Shape of X : (309460, 1001) and Shape of Y : (309460, 12)


### Multi Dimensional Kmers with One Hot Encoding

In [29]:
number_of_unique_kmers = set()
def encode_seq(kmer_token):

    # A 1 0 0 0
    # C 0 1 0 0
    # T/U 0 0 0 1
    # G 0 0 1 0
    # N 0 0 0 0

    encoding_dict = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'U': [0, 0, 0, 1],
        'N': [0, 0, 0, 0],
    }

    encoded_sequence = []
    #print("Input Sequence : ", kmer_token)
    number_of_unique_kmers.add(kmer_token)
    for  base in kmer_token:
        #print("Base : " , base)
        encoded_sequence.append(encoding_dict[base])
    return np.array(encoded_sequence).flatten()


    # bases = 'ACGT'
    # base_dict = {base: i for i, base in enumerate(bases)}
    # one_hot = np.zeros((len(seq), len(bases)))
    # for i, base in enumerate(seq):
    #     if base == 'N':
    #         continue
    #     one_hot[i, base_dict[base]] = 1
    # return one_hot.flatten()

def applyOneHotEncoding(tokenized_sequences):
    encoded_sequences = []
    for seq in tokenized_sequences:
        encoded_sequences.append(encode_seq(seq)) 
 
    return np.array(encoded_sequences)

def applyKmersAndEncoding(seq):
    k=3
    tokens = [seq[i:i+k] for i in range(0, len(seq)-k+1)]
    return tokens
    

def encode_x_with_mult_dimensional_k_mer_one_hot_encoding(x_train_raw):
    truncated_df =  x_train_raw.iloc[:,STRAT_INEDX :END_INDEX] # Window Starts from V501 with 50 window size
    concatenated_column= truncated_df.apply(lambda row: ''.join(map(str, row)), axis=1)
    df_result = truncated_df.assign(Sequence=concatenated_column)
    tokenized_sequences =  df_result['Sequence'].apply(applyKmersAndEncoding).tolist()
    
    # The result, tokenized_sequences, is a list of lists, where each inner list
    #  contains the k-mers of the corresponding RNA sequence from the truncated_df list. 
   
    result = []
    for seq in tokenized_sequences:
        embedding = applyOneHotEncoding(seq)
        result.append(embedding)
    return np.array(result) 



In [30]:
if ENCODING_METHOD == 1:
    x_encoded=encode_x_with_mult_dimensional_k_mer_one_hot_encoding(x_data)

print("Shape of Encoded Data : " ,x_encoded.shape)
print("Sample Records : ", x_encoded[0])  
print("Total Unique K mers : " , len(number_of_unique_kmers))

Shape of Encoded Data :  (309460, 99, 12)
Sample Records :  [[0 0 1 ... 0 1 0]
 [0 0 1 ... 1 0 0]
 [0 0 1 ... 0 1 0]
 ...
 [0 1 0 ... 0 0 1]
 [1 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]
Total Unique K mers :  64


In [31]:
#Decode Y to its Original Form . Additional Class - NonMoD added for Non Modified RNA Sequence 
RMs = ['hAm','hCm','hGm','hTm','hm1A','hm5C','hm5U','hm6A','hm6Am','hm7G','hPsi','Atol','NonMoD']
RMEncoding = [12,1,2,3,4,5,6,7,8,9,10,11,0]


def convert_y_to_original_labels(row):
    label = ""
    for index , n in enumerate(row.tolist()) :
        if n == 1 :
            label = RMs[index]
    if label == '':
        return 'NonMoD'
    
    return label

# Convert One Hot Encoded Y to to Original Labels 
y_original_labels = y_data.apply(convert_y_to_original_labels,axis=1)

if TARGET_ENCODING  == 'LABEL':
    le = LabelEncoder()
    y_encoded = le.fit_transform(y_original_labels)
    y_encoded = torch.tensor(y_encoded, dtype=torch.long)

if TARGET_ENCODING == 'ONE_HOT':
    ohe = OneHotEncoder(sparse=False)
    y_encoded = ohe.fit_transform(y_original_labels.to_numpy().reshape(-1, 1))
    y_encoded = torch.tensor(y_encoded, dtype=torch.float32)
print(y_encoded)

tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]])


c:\Users\shashi.vish\Python Environment\RNA_ModX\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [32]:

# Train set
X_train, X_temp, y_train, y_temp = train_test_split(x_encoded, y_encoded, test_size=0.3, random_state=42)

# Test and Validation set
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)



# Transform into Tensor
if FRAMEWORK == 'PYTORCH':
    X_train = torch.tensor(X_train , dtype=torch.float)
    X_test = torch.tensor(X_test , dtype=torch.float)
    X_valid = torch.tensor(X_valid , dtype=torch.float)

    if TARGET_ENCODING == 'LABEL':
        y_train= torch.tensor(np.array(y_train), dtype=torch.long)  # Cross Entropy Expects Long Data Type
        y_test = torch.tensor(np.array(y_test) , dtype=torch.long)
        y_valid = torch.tensor(np.array(y_valid)  , dtype=torch.long)

    if TARGET_ENCODING == 'ONE_HOT':
        y_train= torch.tensor(np.array(y_train), dtype=torch.float)  # Binary Cross Entropy Expects Float Data Type
        y_test = torch.tensor(np.array(y_test) , dtype=torch.float)
        y_valid = torch.tensor(np.array(y_valid)  , dtype=torch.float)




    print("Shape of X : " , X_train[0].shape)


Shape of X :  torch.Size([99, 12])


In [ ]:
if PERFORM_DATA_BALANCING:
    sm = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
    print(y_resampled.value_counts())

In [33]:
## LSTM Hyper Paramerts 
INPUT_DIMENSION = len(number_of_unique_kmers)
HIDDEN_DIMENSION = 256
BATCH_SIZE = 36
OUTPUT_DIMENSION = 13

### LSTM Model with Cross Entropy Loss (Works with Label Encoder)

In [ ]:
## Define LSTM and Train

class RNADataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

class RNAClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNAClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]
        out = self.fc(last_output)
        return out
         
if MODEL==1 :
    print("Excuting LSTM Model")

    # Create dataset and dataloader


    print("Shape of Tensor : " , X_train.shape)
    train_dataset = RNADataset(X_train, y_train)
    test_dataset = RNADataset(X_test, y_test)

    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Calculating INPUT_DIMENSION for LSTM Mode 
    # If you are just using One Hot then INPUT_DIMENSION would be 4 because there are 4 possible unique nucleosides
    # But if you are using K mers then INPUT_DIMENSION would be number of unique k-mers on all dataset

    # LSTM model Format - (batch_size, sequence_length, input_size)  
    # batch_size comes from DataLoaded , you can tune it.
    # Sequence Length - Lenth of X[0] shape (32, 19, 12)  Here 19 is sequence length , 
    # Input Size = 12 length of X[0][0] - Basically all unique nucleoside or k-mers

    # Initialize the model, loss function, and optimizer
    model = RNAClassifier(input_dim=12, hidden_dim=256, output_dim=13)
    criterion = nn.CrossEntropyLoss()  ## MSELoss of Regression problem 
    optimizer = optim.Adam(model.parameters())

    # Number of Parameters for Model
    total_parameters = []
    for p in model.parameters():
        total_parameters.append(p.numel())
    
    print(f"Total Number of Parameters for Model Training : { sum(total_parameters)} " )

    # Train the model
    num_epochs = 10
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for i, (inputs, labels) in enumerate(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
        epoch_loss = running_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")


### LSTM Model with Binary Cross Entropy Loss (Works with One Hot Encoder)

In [35]:
## Define LSTM and Train

class RNADataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

class RNAClassifier_BCE(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNAClassifier_BCE, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        x = self.fc(hidden.squeeze(0))
        return torch.softmax(x, dim=-1) 
         
if MODEL==1 :
    print("Excuting LSTM Model")

    # Create dataset and dataloader


    print("Shape of Tensor : " , X_train.shape)
    train_dataset = RNADataset(X_train, y_train)
    test_dataset = RNADataset(X_test, y_test)

    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Calculating INPUT_DIMENSION for LSTM Mode 
    # If you are just using One Hot then INPUT_DIMENSION would be 4 because there are 4 possible unique nucleosides
    # But if you are using K mers then INPUT_DIMENSION would be number of unique k-mers on all dataset

    # LSTM model Format - (batch_size, sequence_length, input_size)  
    # batch_size comes from DataLoaded , you can tune it.
    # Sequence Length - Lenth of X[0] shape (32, 19, 12)  Here 19 is sequence length , 
    # Input Size = 12 length of X[0][0] - Basically all unique nucleoside or k-mers

    # Initialize the model, loss function, and optimizer
    model = RNAClassifier_BCE(input_dim=12, hidden_dim=HIDDEN_DIMENSION, output_dim=OUTPUT_DIMENSION)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters())

    # Number of Parameters for Model
    total_parameters = []
    for p in model.parameters():
        total_parameters.append(p.numel())
    
    print(f"Total Number of Parameters for Model Training : { sum(total_parameters)} " )

    # Train the model
    num_epochs = 10
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for i, (inputs, labels) in enumerate(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
        epoch_loss = running_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")




Excuting LSTM Model
Shape of Tensor :  torch.Size([216622, 99, 12])
Total Number of Parameters for Model Training : 279821 
Epoch 1, Loss: 0.1377


In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for x_batch, y_batch in test_dataloader:
        outputs = model(x_batch)
        
        # Basically models predict probability for each class and then finally we pick up with hightest probabiliyt class 
        # and then compare with actual class to calculate accuracy. 
        
        ## Predicted torch.max(outputs, 1) 
#         values=tensor([0.7304, 0.4554, 0.3984, 0.4091, 0.9271, 0.4370, 0.3837, 0.4395, 0.4005,
#         0.5451, 0.5642, 0.6866, 0.5768, 0.5160, 0.5093, 0.4947, 0.6823, 0.4612,
#         0.4073, 0.5041, 0.6982, 0.4116, 0.4092, 0.5589, 0.6010, 0.4693, 0.6324,
#         0.5524, 0.4339, 0.7889, 0.5216, 0.6588]),
#         indices=tensor([ 1, 10, 10, 10,  1, 10, 10, 10,  1, 10,  1,  1,  0,  1,  1, 10,  1, 10,
#         10, 10, 10,  1, 10,  0, 10,  1,  1,  0, 10,  1,  1,  0]))

        _, predicted = torch.max(outputs, 1)  # Get the class index with the highest probability
        print(torch.max(outputs, 1))
        true_labels = torch.max(y_batch, 1).indices  # Get the true class indices from the one-hot encoded labels [0,1,0,0,,,... ] Get index of 1
        correct_predictions += (predicted == true_labels).sum().item()
        total_predictions += y_batch.size(0)

accuracy = correct_predictions / total_predictions


print(f"Validation Accuracy: {accuracy:.4f}")
